# Data cleaning, preprocessing and labeling

**Author:** Bilge Nur Karaca

*Feature names used in this project are altered and does not perfectly represent the original feature names.*

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Get the data
data_df = pd.read_csv("private_dataset.txt", encoding='unicode_escape', sep="\t")

## Filtering data according to pre-defined time periods ruleset

In our dataset, the churn status of customers is absent, so our data is unlabeled. 


The main reason behind that is the non-contractual nature of customer relationship e-commerce context. As vague is the moment at which a customer quits purchasing from a brand, it can be only detected through creation of a ruleset. 

For our project, we will thereby define churn event ourselves. In doing so, we take into consideration the business setting with utmost attention such as consumption cycle of the products, number of repeat purchase per shopper etc.

After careful examination for our dataset, we use the following periods: 
* 9-months prediction audience pool **and** observation,
* 6-months **only** observation period, 
* 6-months churn prediction.

 
Put differently, we take **the customers that have made at least one purchase starting from 15 months ago to 6 months ago**, and we predict their churn situation for **the upcoming 6 months**. 

**We are not interested in the customers that have made a purchase since last 6 months, because their purchase is too recent (i.e. they are active).**

As this ruleset is crucial to **avoid data leakage**, we created relevant variables right away. See below.

In [3]:
# Train data FEATURE period

train_data_feature_period_start_date = pd.to_datetime("2021-03-01 00:00:00")
train_data_feature_period_end_date = pd.to_datetime("2022-06-01 00:00:00") 

# Train data CONTROL period

train_data_control_period_start_date = pd.to_datetime("2022-06-01 00:00:00")
train_data_control_period_end_date = pd.to_datetime("2022-12-01 00:00:00") #

# --------------------------------------------------------

# TEST data FEATURE period

test_data_feature_period_start_date = pd.to_datetime("2021-09-01 00:00:00") 
test_data_feature_period_end_date = pd.to_datetime("2022-12-01 00:00:00")

# TEST data CONTROL period

test_data_control_period_start_date = pd.to_datetime("2022-12-01 00:00:00")
test_data_control_period_end_date = pd.to_datetime("2023-06-01 00:00:00")

# --------------------------------------------------------

train_dataset_global_last_day = "2023-03-01 00:00:00"
test_dataset_global_last_day = "2023-06-01 00:00:00"

# --------------------------------------------------------

# Train data customer-at-risk period

observation_starts = pd.to_datetime('2021-03-01 00:00:00')
observation_ends = pd.to_datetime('2021-12-01 00:00:00')


# Test data customer-at-risk period

observation_starts_test = pd.to_datetime('2021-09-01 00:00:00')
observation_ends_test = pd.to_datetime('2022-06-01 00:00:00')

In [ ]:
# Preserve the initial omnichannel dataset in a variable. The rest is performed on online channel.

omni_data = data_df.copy()

## Fix the glitches in the dataset & detect aggregation needs to achieve the desired dataset format (based on CustomerID index)

1. Filter online channel.
2. Columns names are skipped by 1. Drop 'X_status' from columns names and ve add "to-be-removed"(tbr) column at the end.
3. Remove "tbr" column. Drop columns which have only 1 unique value.
4. Detected 2 test users. Remove 2 test users & their corresponding rows.
5. Create dataframes separately for customer ve transaction features.
6. Group the customer dataframe by aggregating fields using "last" function.
7. Group the transaction dataframe by aggregating fields.
8. Replace the Turkish characters in location names.
9. Replace misspellings in the City values.
10. Examine net amount or price == 0 cases.
11. Remove col7 ve col8 due to high % of null values.

### 1. Filter online channel data

In [ ]:
# Filter online

data_df = data_df[data_df.ChannelName == "ONLINE CHANNEL"]

print(data_df[["ChannelName"]].value_counts())
print(data_df.shape[0])
print(data_df.isna().sum()/data_df.shape[0])
data_df.info()

### 2. Columns names are skipped by 1. Drop 'X_status' from columns names and add "to-be-removed"(tbr) column at the end.


In [ ]:
new_cols = ["Insert", "the","list", "of", "column", "names", "here"]
data_df.columns = new_cols
data_df.columns 

### 3. Remove "tbr" column. Drop columns which have only 1 unique value. Remove 'ProductID','Phone', 'Email'.

In [ ]:
# Remove columns that have only 1 unique value or all null

unique_val_cols = []
for col in data_df.columns:
    if len(data_df[col].unique()) <= 1: 
        print(col)
        unique_val_cols.append(col)
print(unique_val_cols)

data_df.drop(unique_val_cols, axis=1,inplace=True)

# Drop columns that does not have value: - Phone and Email unique // col7 and col8 are %80 null. 

data_df.drop(['ProductID','Phone', 'Email', 'Email2', 'col7', 'col8'], axis=1,inplace=True) 

### 4. Investigate suspicious users. Remove 2 test users & their rows.



In [ ]:
# Checking unique city value counts. Detecting more unique city names than those which exist.

data_df[["CustomerID", "City"]].groupby("CustomerID").nunique().sort_values(by="City")

# Checking the user w/ impossible number of unique cities.
# TEST USER 1

print(data_df["City"][data_df["CustomerID"]== "{HIDDEN-CUSTOMER-ID}"].unique())

# TEST USER 2: too many cities.  

data_df["City"][data_df["CustomerID"]== "{HIDDEN-CUSTOMER-ID2}"].unique()

In [ ]:
# more investigation 

test_user1 = "{HIDDEN-CUSTOMER-ID}"
data_df[["CustomerID","City","NetAmount"]][data_df["CustomerID"] == test_user1]

In [ ]:
# more investigation 

test_user2 = "{HIDDEN-CUSTOMER-ID2}"
data_df[["CustomerID","City","NetAmount","Price"]][data_df["CustomerID"] == 
                                                   test_user2].sort_values(by="Price")

In [ ]:
# more investigation 

print("Raws inc. test_user_1:", len(list(data_df["CustomerID"][(data_df["CustomerID"]==test_user1)])))
print("Raws inc. test_user_2:",len(list(data_df["CustomerID"][(data_df["CustomerID"]==test_user2)])))

test_user_idx = list(data_df["CustomerID"][(data_df["CustomerID"]==test_user1)
                                           |(data_df["CustomerID"]==test_user2)]
                     .index)
print("Raws inc. all test users:",len(test_user_idx))

In [ ]:
# Drop test users

data_df.drop(test_user_idx, axis=0, inplace=True)
data_df.shape

# Reset index & drop index column

data_df.reset_index(inplace=True, drop=True)

### 5. Replace the Turkish characters in location names.

In [ ]:
# Turkish char adjustments

data_df['col1'] = data_df['col1'].str.replace("Ý","I").str.replace("ý","i").str.replace("ð","g").str.replace("þ","s").str.replace("Þ","S")
data_df['col2'] = data_df['col2'].str.replace("Ý","I").str.replace("ý","i").str.replace("ð","g").str.replace("þ","s").str.replace("Þ","S")
data_df['col3'] = data_df['col3'].str.replace("Ý","I").str.replace("ý","i").str.replace("ð","g").str.replace("þ","s").str.replace("Þ","S")
data_df['col4'] = data_df['col4'].str.replace("Ý","I").str.replace("ý","i").str.replace("ð","g").str.replace("þ","s").str.replace("Þ","S")
data_df['col5'] = data_df['col5'].str.replace("Ý","I").str.replace("ý","i").str.replace("ð","g").str.replace("þ","s").str.replace("Þ","S")
data_df['col5'] = data_df['col6'].str.replace("Ý","I").str.replace("ý","i").str.replace("ð","g").str.replace("þ","s").str.replace("Þ","S")

# City name adjustments

data_df[["col1","col2","col3","col4","col5","col6"]].head()

### 6. Replace null values of CouponType as "no_coupon"

In [ ]:
data_df["CouponType"].fillna("no_coupon_used",inplace=True)

### 7. Remove missing values of City1 by filling from City2 as long as District1 matches District2.

In [ ]:
# Fill out missing values of City1 by City2 if districts of both match.

pd.options.display.max_columns=40

data_df["City1"] = np.where((data_df["City2"]!= data_df["City1"]) & 
                                     (data_df["District2"] != data_df["Distric1"]) &
                                     (data_df["City1"].apply(str).apply(len) > 15),
                                        "flag",data_df["City2"])

### 8. Examine products with Price == 0 

In [ ]:
cust_with_0_idx = list(data_df[["CustomerID"]][data_df["Price"] == 0].index)
cust_with_0_custID = list(data_df["CustomerID"].iloc[cust_with_0_idx].unique())

cust_with_0_agg = data_df[["CustomerID","DiscountAmount","NetAmount"]][(data_df["CustomerID"]
                                                .isin(cust_with_0_custID))].groupby("CustomerID").sum().sort_values(by="NetAmount")

cust_with_0_agg


### 9. Datetime conversion

In [ ]:
data_df["BirthDate"] = pd.to_datetime(data_df["BirthDate"], format="%Y-%m-%d %H:%M:%S" )
data_df["TransactionDate"] = pd.to_datetime(data_df["TransactionDate"], format="%Y-%m-%d %H:%M:%S" )


### 10. Create # of days features

In [ ]:
# Define a function to get number of days since the input date

# global_last_day = "2023-06-17 15:00:00"

def days_since(last_tranaction_date_of_customer, dataset_global_last_day):
    dataset_global_last = pd.to_datetime(dataset_global_last_day, format="%Y-%m-%d %H:%M:%S" )
    days_since= dataset_global_last - last_tranaction_date_of_customer
    days_since =days_since.days
    return days_since

data_df["TransactionDate"].apply(days_since)

### 11. Add an offline-channel transaction number feature

This feature will enable the model to learn offline behavior of the customers.

In [ ]:
# get unique transactions per customer

omni_data = omni_data.drop_duplicates(subset = ["CustomerID", "TransactionDate"])

# datetime conversion

omni_data["TransactionDate"] = pd.to_datetime(omni_data["TransactionDate"], format="%Y-%m-%d %H:%M:%S" )

# Filter according to Train data FEATURE period

omni_data_feature_period_train = omni_data[(omni_data["TransactionDate"]> train_data_feature_period_start_date) &
                                                      (omni_data["TransactionDate"]< train_data_feature_period_end_date)].reset_index(drop=True)


# Filter according to TEST data FEATURE period


omni_data_feature_period_test = omni_data[(omni_data["TransactionDate"]> test_data_feature_period_start_date) &
                                                      (omni_data["TransactionDate"]< test_data_feature_period_end_date)].reset_index(drop=True)

# Group by Customer ID to get transaction number

omnichannel_trx_num_train = omni_data_feature_period_train[["CustomerID", 
                                                            "TransactionDate"]].groupby("CustomerID").count()

omnichannel_trx_num_test = omni_data_feature_period_test[["CustomerID", 
                                                            "TransactionDate"]].groupby("CustomerID").count()

# Get online transaction number

online_trx_num_train = omni_data_feature_period_train[["CustomerID", 
                                                         "TransactionDate"
                                                        ]
                               ][omni_data_feature_period_train.ChannelName == "ONLINE CHANNEL"].groupby("CustomerID").count()

online_trx_num_test = omni_data_feature_period_test[["CustomerID", 
                                                     "TransactionDate"
                                                    ]
                               ][omni_data_feature_period_test.ChannelName == "ONLINE CHANNEL"].groupby("CustomerID").count()


# Merge online & omnichannel purchase number 

trx_merged_train = online_trx_num_train.merge(omnichannel_trx_num_train, how="left", on="CustomerID")
trx_merged_test  = online_trx_num_test.merge(omnichannel_trx_num_test, how="left", on="CustomerID")

# Get offline number by substraction

trx_merged_train["offline"] = trx_merged_train["TransactionDate_y"] - trx_merged_train["TransactionDate_x"]
trx_merged_test["offline"] = trx_merged_test["TransactionDate_y"] - trx_merged_test["TransactionDate_x"]


offline_purchase_train = trx_merged_train[["offline"]].reset_index()
offline_purchase_test = trx_merged_test[["offline"]].reset_index()

# Add these to the dataframe

offline_purchase_train
offline_purchase_test

In [6]:
train_feature_period_length = print((train_data_feature_period_end_date-train_data_feature_period_start_date))
train_control_period_length = print((train_data_control_period_end_date-train_data_control_period_start_date))

457 days 00:00:00
183 days 00:00:00


## 12. Create customer-based and order-based dataframes

## SPLIT DATA FOR TRAIN & TEST

In [ ]:
# Split the transaction data according to dates

train_df = data_df[(data_df["TransactionDate"]> train_data_feature_period_start_date) &
                    (data_df["TransactionDate"]< train_data_feature_period_end_date)].reset_index(drop=True) #sadece featurelar


# --------------------------------------------------------

test_df = data_df[(data_df["TransactionDate"]> test_data_feature_period_start_date) &
                    (data_df["TransactionDate"]< test_data_feature_period_end_date)].reset_index(drop=True)

print(train_data_feature_period_end_date-train_data_feature_period_start_date)
print(test_data_feature_period_end_date-test_data_feature_period_start_date)

In [ ]:
# For TRAIN data, slice the transaction date of the observation (pooling) period

df_train_only_for_observation_period = data_df[(data_df["TransactionDate"]> observation_starts) &
                                                      (data_df["TransactionDate"]< observation_ends)].reset_index(drop=True)

# We only look for which customers has made a purchase within this period.
# Get labels for that.

observation_labels = df_train_only_for_observation_period.groupby("CustomerID").sum()
observation_labels["label"] = 1
observation_labels.drop(["TransactionDate_code",
                         "Price",
                         "Quantity",
                         "Amount",
                         "Discount_Amount",
                         "Net_Amount",
                         "Basket_Size"], axis=1, inplace=True)
observation_labels

In [ ]:
# For TEST data, slice the transaction data of the observation (pooling) period

df_test_only_for_observation_period = data_df[(data_df["TransactionDate"]> observation_starts_test) &
                                                      (data_df["TransactionDate"]< observation_ends_test)].reset_index(drop=True)

# We only look for which customers has made a purchase within this period.
# Get labels for that

observation_labels_test = df_test_only_for_observation_period.groupby("CustomerID").sum()
observation_labels_test["label"] = 1
observation_labels_test.drop(["TransactionDate_code",
                         "Price",
                         "Quantity",
                         "Amount",
                         "Discount_Amount",
                         "Net_Amount",
                         "Basket_Size"], axis=1, inplace=True)
observation_labels_test

### CREATE ORDER-BASED-FEATURES DATAFRAME / Unique identifier is InvoiceID.

In [ ]:
# Put together values that occur per purchase and that stay same within the detail lines of a given pruchse

def func_transform1(data_df):
    bask_data_part1=  data_df[["CustomerID",
                                "InvoiceID",
                                "Invoice_Type",
                                "TransactionDate",
                                "City2",
                                "CampaignCouponCode",
                                "City1",
                                "Device",
                                #"Discount_Amount",
                                #"Net_Amount",
                                "Basket_Size"]].drop_duplicates(subset=["InvoiceID"])
    return bask_data_part1

# aggregate values per purchase. Those are features that change across different lines of a pruchase.

def func_transform2(data_df):
    bask_data_part2 = data_df[[ "InvoiceID",
                                "Discount_Amount",
                                "Net_Amount"]].groupby("InvoiceID").agg({'Discount_Amount': ['sum'],
                                                                        'Net_Amount': ['sum']})
    return bask_data_part2

# merge two subsets of basket_data

def func_transform3(bask_data_part1, bask_data_part2):
    bask_data = bask_data_part1.merge(bask_data_part2, on="InvoiceID", how="outer")
    return bask_data

# create a bool column to see whether there's a change between two addresses

def func_transform4(bask_data):
    bask_data["isShippedToBilled"] = np.where(bask_data["City1"] == bask_data["City2"],1,0)
    return

# drop city1 columns

def func_transform5(bask_data):
    bask_data.drop("City1",axis=1,inplace=True)
    return

# calculate days since the invoice is processed

def func_transform6(bask_data, dataset_global_last_d):
    bask_data["days_since_transaction_occurred"] = bask_data["TransactionDate"].apply(lambda x: days_since(x, dataset_global_last_d))                                                                                             
    return


In [ ]:
# APPLY TRANSFORMATION FUNCTIONS

# Put together values that occur per purchase and that stay same within the detail lines of a given pruchse
bask_data_part1_train = func_transform1(train_df)
bask_data_part1_test = func_transform1(test_df)

# Aggregate values per purchase. Those are features that change across different lines of a pruchase.
bask_data_part2_train = func_transform2(train_df)
bask_data_part2_test = func_transform2(test_df)

# Merge two subsets of bask_data
bask_data_train = func_transform3(bask_data_part1_train, bask_data_part2_train)
bask_data_test = func_transform3(bask_data_part1_test, bask_data_part2_test)

# Create a bool column to see whether there's a change between billing and shipping address
func_transform4(bask_data_train)
func_transform4(bask_data_test)

# Drop shipping city
func_transform5(bask_data_train)
func_transform5(bask_data_test)

# Calculate days since the invoice is processed
func_transform6(bask_data_train, train_dataset_global_last_day)
func_transform6(bask_data_test, test_dataset_global_last_day)


### CREATE CUSTOMER-BASED-FEATURES DATAFRAME / Unique identifier is CustomerID.

In [ ]:
cust_features = ['CustomerID', 'BirthDate','isContactable']


def func_transform7(data_df, cust_features):
    cust_data = pd.pivot_table(data=data_df, 
                               index=data_df["CustomerID"],
                               values = data_df[cust_features],
                               aggfunc='last') # last value is the most updated
    cust_data.reset_index(drop=True,inplace=True)
    return cust_data

cust_data_train = func_transform7(train_df, cust_features)
cust_data_test = func_transform7(test_df, cust_features)

cust_data_train.isna().sum()

### JOIN 1: CUSTOMER-BASED-DATA + ORDER-BASED-DATA


In [ ]:
def func_transform8(cust_data, bask_data):
    bask_cust_data = cust_data.merge(bask_data, on="CustomerID", how="outer")
    
    bask_cust_data.columns = ["insert", "column", "names", "here"]
    
    return bask_cust_data

bask_cust_data_train = func_transform8(cust_data_train, bask_data_train)
bask_cust_data_test = func_transform8(cust_data_test, bask_data_test)

In [ ]:
bask_cust_data_train[bask_cust_data_train["CustomerID"].isna()==False]

In [ ]:
def func_transform9(bask_cust_data):
    
    df =    bask_cust_data[
                [
                    "CustomerID",
                    "BirthDate",
                    "isContactable",
                    "City",
                    "InvoiceID",
                    "CouponType",
                    "Device",
                    "Basket_Size",
                    "Discount_Amount_sum",
                    "Net_Amount_sum",
                    'isShippedToBilled',
                    'days_since_transaction_occured']].groupby("CustomerID").agg(
                {
                    'BirthDate': ['last'], 
                    'isContactable': ['last'], 
                    'City':['last'], 
                    'InvoiceID':['count'], # total number of orders
                    'CouponType': ['last'], #total times a coupon is used
                    'Basket_Size':[np.mean, sum, 'min', 'max'], # 
                    "Discount_Amount_sum": [np.mean, sum,'count'], # how many times a discount is used etc.
                    "Net_Amount_sum": [np.mean, sum], #AOV, total moneraty value
                    'isShippedToBilled': [sum],
                    'days_since_transaction_occured': ['min', 'max'] # days since first & last order
                })


    df.columns = ['BirthDate',
                   'isContactable',
                    'City_last',
                    'InvoiceID_count',
                    'CouponType_last',
                    'Basket_Size_mean',
                    'Basket_Size_sum',
                    'Basket_Size_min',
                    'Basket_Size_max',
                    'Discount_Amount_sum_mean',
                    'Discount_Amount_sum_sum',
                    'Discount_Amount_sum_count',
                    'Net_Amount_sum_mean',
                    'Net_Amount_sum_sum',
                    'isShippedToBilled_sum',
                    'days_since_last_purchase',
                    'days_since_first_purchase']  
    return df

In [ ]:
df_train = func_transform9(bask_cust_data_train)
df_test = func_transform9(bask_cust_data_test)

#### Add "number_of_returns" feature to df

In [ ]:
def func_transform10(bask_data, df):

    # get number of returns by aggregation
    num_returns = bask_data[["CustomerID","Invoice_Type"]][bask_data["Invoice_Type"]=="Return"].groupby("CustomerID").count()

    # merge number of returns column with df
    df = df.merge(num_returns, on ="CustomerID", how="left")

    # fill na with 0 (since no returns happened)
    df["Invoice_Type"].fillna(0,inplace=True)

    # rename the column
    df.rename(columns={"Invoice_Type": "num_returns"},inplace=True)
    
    return df

df_train = func_transform10(bask_data_train, df_train)
df_test = func_transform10(bask_data_test, df_test)

#### Add device_type to df (first)

In [ ]:
#pd.options.display.min_rows=50

def func_transform11(bask_data, df):
    device_most_recent = bask_data.sort_values(by=
                                                 "days_since_transaction_occurred")[["CustomerID", 
                                                                                     "Device"
                                                                                    ]
                                                                                ].groupby("CustomerID").agg(
                                                                                    {
                                                                                     "Device": 'first'
                                                                                    }
    )


    df = df.merge(device_most_recent,on="CustomerID", how="left")
    return df

df_train = func_transform11(bask_data_train, df_train)
df_test = func_transform11(bask_data_test, df_test)

In [ ]:
def func_transform11(data_df, df):
    
    # Compute frequencies for each axe per customer (how many products from each axe)

    ct_axe = pd.crosstab(data_df["CustomerID"], data_df["Product_Category"])

    # Merge with the dataset

    df = df.merge(ct_axe,how="left",on="CustomerID")
    
    return df

#---------------------

def func_transform12(data_df, df):
    # Compute frequencies for each axe per customer (how many products from each axe)

    ct_grp = pd.crosstab(data_df["CustomerID"], data_df["Product_Axe"])

    # Merge with the dataset

    df= df.merge(ct_grp,how="left",on="CustomerID")
    
    return df


df_train = func_transform11(train_df, df_train)
df_test = func_transform11(test_df, df_test)

df_train = func_transform12(train_df, df_train)
df_test = func_transform12(test_df, df_test)

In [ ]:
most_recent_return = bask_data[["CustomerID",
                                "TransactionDate"]
                               ][bask_data["Invoice_Type"]=="Return"].groupby("CustomerID").max()

most_recent_return["days_since_last_return"] = most_recent_return["TransactionDate"].apply(days_since_last_purchase)
most_recent_return.drop("TransactionDate", axis=1, inplace=True)
most_recent_return["days_since_last_return"]

In [ ]:
def func_transform13(df, dataset_global_last_d):
    df["Age"] = df["BirthDate"].apply(lambda x: days_since(x, dataset_global_last_d))
    df["Age"] = df["Age"] / 365
    df.drop("BirthDate",inplace=True, axis=1)
    return df


df_train = func_transform13(df_train, train_dataset_global_last_day)
df_test = func_transform13(df_test, test_dataset_global_last_day)

In [ ]:
# add offline

df_train = df_train.merge(offline_purchase_train,on="CustomerID",how="left")
df_test = df_test.merge(offline_purchase_test,on="CustomerID",how="left")

# Train Labeling

In [ ]:
# create a transaction-based dataframe that covers only the control period

control_group_df = data_df[(data_df["TransactionDate"] >= pd.to_datetime(train_data_control_period_start_date)) & 
                        (data_df["TransactionDate"] < pd.to_datetime(train_data_control_period_end_date))]


# label those as "not a churn"

control_group_df["label"]=0

# take only necessary columns

control_group_df = control_group_df[["CustomerID","label"]]

# unify customers

control_group_df = control_group_df.groupby("CustomerID").sum()

# Rename the "label" column in distinctive way

observation_labels["observation_label"] = 1
observation_labels.drop(["label"],axis=1,inplace=True)

# Merge control group and observation. We want all the observation list.

label_prep = observation_labels.merge(control_group_df, on="CustomerID", how="left")
label_prep.shape

In [ ]:
label_prep.drop("observation_label", axis=1, inplace=True)

In [ ]:
print(control_group_df.shape)
print(observation_labels.shape)
print(train_data_control_period_end_date-train_data_control_period_start_date)
print(test_data_control_period_end_date-test_data_control_period_start_date)

In [ ]:
label_prep.label.value_counts()

In [ ]:
label_prep["label"].fillna(1, inplace=True)
label_prep.label.value_counts()

In [ ]:
print(df_train.shape)
print(df_test.shape)
print(label_prep.shape)

In [ ]:
train_data = df_train.merge(label_prep, on="CustomerID", how="left")

# only customers that we selected have labels. To drop other customers, remove when label = null

train_data = train_data[train_data["label"].isna()==False]

print("Labeling is done.")

### Test labeling

In [ ]:
# create a transaction-based dataframe that covers only the control period

control_group_df_test = data_df[(data_df["TransactionDate"] >= pd.to_datetime(test_data_control_period_start_date)) & 
                        (data_df["TransactionDate"] < pd.to_datetime(test_data_control_period_end_date))]


# label those as "not a churn"

control_group_df_test["label"]=0

# take only necessary columns

control_group_df_test = control_group_df_test[["CustomerID","label"]]

# unify customers

control_group_df_test = control_group_df_test.groupby("CustomerID").sum()

# Rename the "label" column in distinctive way

observation_labels_test["observation_label"] = 1
observation_labels_test.drop(["label"],axis=1,inplace=True)

# Merge control group and observation. We want all the observation list.

label_prep_test = observation_labels_test.merge(control_group_df_test, on="CustomerID", how="left")
label_prep_test.shape


In [ ]:
label_prep_test.drop("observation_label", axis=1, inplace=True)

In [ ]:
print(control_group_df_test.shape)
print(observation_labels_test.shape)

In [ ]:
label_prep_test["label"].fillna(1, inplace=True)
label_prep_test.label.value_counts()

In [ ]:
# merge labels & features

test_data = df_test.merge(label_prep_test, on="CustomerID", how="left")

# only customers that we selected have labels. To drop other customers, remove when label = null

test_data = test_data[test_data["label"].isna()==False]

#### Add purchase_freq feature

In [ ]:
train_data["purchase_freq"] = (train_data["days_since_first_purchase"
                                         ]-train_data["days_since_last_purchase"
                                                     ])/train_data['num_of_transactions']

In [ ]:
test_data["purchase_freq"] = (test_data["days_since_first_purchase"
                                         ]-test_data["days_since_last_purchase"
                                                     ])/test_data['num_of_transactions']

## EXPORT

In [ ]:
train_data_df = train_data.reset_index()
import pickle 
f = open("OM_D1_train_data", 'wb') 
pickle.dump(train_data_df, f)
f.close()

test_data_df = test_data.reset_index()
import pickle 
f = open("OM_D1_test_data", 'wb') 
pickle.dump(test_data_df, f)
f.close()

# Dataset export for networkx graph

In [ ]:
train_data_feature_period_end_date-train_data_feature_period_start_date

In [ ]:
# Split the transaction data according to dates

train_df_nx = data_df[(data_df["TransactionDate"]> train_data_feature_period_start_date) &
                    (data_df["TransactionDate"]< train_data_feature_period_end_date)].reset_index(drop=True) #sadece featurelar


# --------------------------------------------------------



test_df_nx = data_df[(data_df["TransactionDate"]> test_data_feature_period_start_date) &
                    (data_df["TransactionDate"]< test_data_feature_period_end_date)].reset_index(drop=True)



In [ ]:
train_df_nx.groupby("CustomerID").count()

In [ ]:
train_df_nx[["Ean","ProductCode"]]

In [ ]:
# Take ean and customerID columns to build a network

data_nx_NF = train_df_nx.copy()
data_nx_NF = data_nx_NF[["CustomerID", "Ean"]]

# Drop null values

data_nx_NF.dropna(inplace=True)
data_nx_NF

In [ ]:
# Convert EAN values to appropriate dtype

import numpy as np

def convert_to_int(x):
    try:
        return int(x)
    except (ValueError, TypeError):
        return x

data_nx_NF["Ean"] = data_nx_NF["Ean"].apply(convert_to_int)
data_nx_NF["Ean"] = data_nx_NF["Ean"].transform(str)
data_nx_NF["Ean"]

In [ ]:
data_nx_NF.groupby("CustomerID").count()

In [ ]:
data_nx_NF.reset_index(inplace=True, drop=True)
import pickle 
fl = open("OM_D3_dataset-network", 'wb') 
pickle.dump(data_nx_NF, fl)
fl.close()